In [ ]:
from bs4 import BeautifulSoup
import requests

#url = 'https://www.tekstowo.pl/artysci_na,A,strona,1.html'
pages = list(range(1, 11))

domain = 'https://tekstowo.pl'
urls = []
cnt = 0

# Extracting the artist list
for page in pages:
	url = f'https://www.tekstowo.pl/artysci_na,A,strona,{page}.html'
	try:
		req = requests.get(url)
		if req.ok:
			soup = BeautifulSoup(req.content, 'lxml')
			for link in soup.find_all('a'):
				item = link.get('href')
				#if type(link.get('href')) == str and 'piosenki_' in link.get('href'):
				if isinstance(item, str) and 'piosenki_' in item:
					urls.append(link.get('href'))

			#print(soup.prettify())
			#art = soup.find_all(class_='title')

			#print(art)

	except Exception as e:
		cnt += 1
		print(e)

print(len(urls))

In [162]:
def get_pages_number(url: str) -> int:
	"""
	Get the highest page number per given letter.
	"""
	max_page = 0

	req = requests.get(url)
	if req.ok:
		soup = BeautifulSoup(req.content, 'lxml')
		for page in soup.find_all(class_='page-link'):
			if page.text.isnumeric() and int(page.text) > max_page:
				max_page = int(page.text)
	#print(f"Max page: {max_page}")
	return max_page

In [163]:
import string
import time

def create_lut_pagination():
	"""
	Creates a look-up table for each letter containing max page number
	"""

	alphabet = list(string.ascii_uppercase) + ['pozostale']

	lut_pages = {}

	for letter in alphabet:
			time.sleep(5)
			url = f"https://www.tekstowo.pl/artysci_na,{letter}.html"
			lut_pages[letter] = get_pages_number(url)
			print(f"Letter {letter} has {lut_pages[letter]} pages of artists.")	
	
	return lut_pages

In [171]:
import time

def get_artists(letter, max_page_per_letter):
	"""
	Scrape all of the artists starting with a given letter in the alphabet.
	"""
	
	urls = []
	limit = max_page_per_letter[letter]+1

	for page in range(1, limit):

		url = f"https://www.tekstowo.pl/artysci_na,{letter},strona,{page}.html"

		try:
			time.sleep(5)
			response = requests.get(url)
			if response.ok:
				soup = BeautifulSoup(response.content, 'lxml')
				for link in soup.find_all('a'):
					item = link.get('href')
					if type(item) == str and 'piosenki_' in item:
						urls.append('https://tekstowo.pl' + item)
		except Exception as e:
			print(e)
		print(f"{letter}: Visited {page}/{limit-1}")
		
	print(f"Letter {letter}: collected {len(urls)} artists")
	return urls, len(urls)

In [172]:
import requests
import time

def get_artist_songs(artist_url):
    urls = []
    processed_first_page = False

    while not processed_first_page:
        time.sleep(5)
        try:
            response = requests.get(artist_url)
            if response.ok:
                soup = BeautifulSoup(response.content, 'html.parser')
                songs = soup.find_all(class_='box-przeboje')
                artist = soup.find(class_="col-md-7 col-lg-8 px-0")
                artist = artist.text.split(" (")[0].strip()
                
                for song in songs:
                    song_title_element = song.find(class_='title')

                    if song_title_element:
                        if artist in song_title_element.text.strip():
                            song_url = "https://tekstowo.pl" + song_title_element['href']
                            if not ".plpiosenka" in song_url and song_url not in urls:
                                urls.append(song_url)
                    else:
                        pass

                button_next_page = soup.find_all(class_='page-link')
                if button_next_page and len(button_next_page) > 0:
                    button_next_page = button_next_page[-1]

                    if 'następna' in button_next_page.text.lower():
                        artist_url = "https://tekstowo.pl" + button_next_page['href']
                    else:
                        break
                else:
                    break

        except Exception as e:
            print(e)
            
    print(artist)
    print(f"Collected {len(urls)} song URLs")
    return urls

In [166]:
import requests

def extract_song(song_url):
	song = ""
	song_translation = None

	try:
		response = requests.get(song_url)
		if response.ok:
			soup = BeautifulSoup(response.content, 'lxml')

			# Song title
			song_title = soup.find(class_='col-lg-7').text.strip()
		
			# Original song
			song_html = soup.find(class_='inner-text')			
			song = song_html.text.strip()

			# Translated version
			transl_html = soup.find('div', id='translation')
			song_translation = transl_html.text.strip()

	except Exception as e:
		print(e)

	return song, song_translation, song_title

In [167]:
import langdetect

def assess_language(song_text):
	if not isinstance(song_text, str):
		return False
	
	#elif len(song_text) == 0:
		#raise TypeError("Empty file")

	else:
		try:
			return langdetect.detect(song_text)
		except langdetect.lang_detect_exception.LangDetectException:
			return "Language detection failed"

In [168]:
import os

def save_songs(title, song_1, song_2, lang_1, lang_2, letter):
	"""
	Save the scraped song with corresponding languages
	and title.
	"""

	save_dir = "teksty/"

	clean_title = title.replace('/', '-')
	
	song_1_filename = f"{letter}/{clean_title}__{lang_1.upper()}.txt"
	song_2_filename = f"{letter}/{clean_title}__TRAN__{lang_2.upper()}.txt"

	with open(os.path.join(save_dir, song_1_filename), 'w', encoding='utf-8') as f:
		f.write(song_1)
		f.close()	
		print(f"Original successfully saved: {title}")

	if len(lang_2) < 3 or song_2 == 0:
		with open(os.path.join(save_dir, song_2_filename), 'w', encoding='utf-8') as f:
			f.write(song_2)
			f.close
			print(f"Translation successfully saved: {title}")
	
	else:
		print("No translation found")


In [169]:
alphabet = list(string.ascii_lowercase.upper()) + ["pozostale"]
print(alphabet)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'pozostale']


In [ ]:
max_page_per_letter = create_lut_pagination()

for letter in alphabet:
    cnt = 0

    artist_urls, artist_cnt = get_artists(letter, max_page_per_letter)
    print(f"Letter {letter} artists collected.")

    for artist_url in artist_urls:
        artist_songs = get_artist_songs(artist_url)
        #print(f"Artist")

        try:
            for artist_song in artist_songs:
                text1, text2, title = extract_song(artist_song)
                lang1 = assess_language(text1)
                lang2 = assess_language(text2)

                save_songs(title, text1, text2, lang1, lang2)
        except Exception as e:
            print(e)
        
        cnt += 1
    
    print(f"Letter {letter}: processed {cnt}/{artist_cnt}")
    

In [173]:
#letter = "Q"

for letter in alphabet:
	
	cnt = 0

	artist_urls, artist_cnt = get_artists(letter, max_page_per_letter)
	print(f"Letter {letter} artists collected.")


		
	for artist_url in artist_urls:
		print(artist_url)
		artist_songs = get_artist_songs(artist_url)

		try:
			time.sleep(5)
			for artist_song in artist_songs:
				text1, text2, title = extract_song(artist_song)
				
				if len(text1) > 10:
					lang1 = assess_language(text1)
				if len(text2) > 10:
					lang2 = assess_language(text2)

				save_songs(title, text1, text2, lang1, lang2, letter)

		except Exception as e:
			print(e)
		
	cnt += 1

	print(f"Letter {letter}: procesed {cnt}/{artist_cnt}")

A: Visited 1/539
A: Visited 2/539
